<a href="https://colab.research.google.com/github/polaroidz/polaroidz/blob/main/Pandas_Handbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np


# Aggregation Group By

In [52]:
df = sns.load_dataset('titanic')

In [ ]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [ ]:
df['fare'].agg(['sum', 'mean'])

sum     28693.949300
mean       32.204208
Name: fare, dtype: float64

In [ ]:
df.agg({
    'fare': ['sum', 'mean'],
    'sex': ['count']
})

,fare,sex
count,NaN,891.0
mean,32.204208,NaN
sum,28693.949300,NaN


In [ ]:
df.agg(
    fare_sum=('fare', 'sum'),
    fare_mean=('fare', 'mean'),
    sex_count=('sex', 'count')
)

,fare,sex
fare_sum,28693.949300,NaN
fare_mean,32.204208,NaN
sex_count,NaN,891.0


In [ ]:
df.groupby(['embark_town']).agg({
    'fare': ['sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad', 'prod']
}).round(2)

fare                      ...                                      
                  sum   mean median   min  ...    std      var    mad           prod
embark_town                                ...                                      
Cherbourg    10072.30  59.95  29.70  4.01  ...  83.91  7041.39  53.02  6.193716e+250
Queenstown    1022.25  13.28   7.75  6.75  ...  14.19   201.30   7.87   6.458671e+78
Southampton  17439.40  27.08  13.00  0.00  ...  35.89  1287.95  21.30   0.000000e+00

[3 rows x 9 columns]

In [ ]:
df.groupby(['embark_town']).agg({
    'fare': ['describe']
})

fare                        ...                            
            describe                        ...                            
               count       mean        std  ...    50%        75%       max
embark_town                                 ...                            
Cherbourg      168.0  59.954144  83.912994  ...  29.70  78.500025  512.3292
Queenstown      77.0  13.276030  14.188047  ...   7.75  15.500000   90.0000
Southampton    644.0  27.079812  35.887993  ...  13.00  27.900000  263.0000

[3 rows x 8 columns]

In [ ]:
df.groupby(['deck']).agg({
    'embark_town': ['count', 'nunique', 'size']
})

embark_town             
           count nunique size
deck                         
A             15       2   15
B             45       2   47
C             59       3   59
D             33       2   33
E             32       3   32
F             13       3   13
G              4       1    4

In [ ]:
df.sort_values(by=['fare'], ascending=False) \
  .groupby(['embark_town']) \
  .agg({
      'fare': ['first', 'last'] # max, min
  })

fare        
                first    last
embark_town                  
Cherbourg    512.3292  4.0125
Queenstown    90.0000  6.7500
Southampton  263.0000  0.0000

In [ ]:
df.sort_values(by=['fare'], ascending=False) \
  .groupby(['embark_town']) \
  .agg({
      'fare': ['max', 'min'] # max, min
  })

fare        
                  max     min
embark_town                  
Cherbourg    512.3292  4.0125
Queenstown    90.0000  6.7500
Southampton  263.0000  0.0000

In [ ]:
df.sort_values(by=['fare'], ascending=False) \
  .groupby(['embark_town']) \
  .agg({
      'fare': ['idxmax', 'idxmin']
  })

fare       
            idxmax idxmin
embark_town              
Cherbourg      258    378
Queenstown     412    654
Southampton     88    466

In [ ]:
df.loc[[258, 378]]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
378,0,3,male,20.0,0,0,4.0125,C,Third,man,True,NaN,Cherbourg,no,True


In [ ]:
# shortcut
df.loc[df.groupby('class')['fare'].idxmax()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
72,0,2,male,21.0,0,0,73.5000,S,Second,man,True,NaN,Southampton,no,True
159,0,3,male,NaN,8,2,69.5500,S,Third,man,True,NaN,Southampton,no,False


In [ ]:
from scipy.stats import skew, mode

df.groupby(['embark_town']).agg({
    'fare': [skew, mode, pd.Series.mode]
})

fare                          
                 skew              mode    mode
embark_town                                    
Cherbourg    3.305112  ([7.2292], [15])  7.2292
Queenstown   4.265111    ([7.75], [30])  7.7500
Southampton  3.640276    ([8.05], [43])  8.0500

In [ ]:
df.groupby(['class']).agg({
    'deck': ['nunique', mode, set]
})

deck                                   
       nunique         mode                   set
class                                            
First        5  ([C], [59])  {nan, C, D, E, A, B}
Second       3   ([F], [8])        {nan, E, D, F}
Third        3   ([F], [5])        {nan, G, E, F}

In [ ]:
# custom agg functions

from functools import partial

q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '25%'

# ou

def percentile_25(x):
  return x.quantile(.25)

# ou

lambda_25 = lambda x: x.quantile(.25)
lambda_25.__name__ = 'lambda_25%'

In [ ]:
df.groupby(['embark_town']).agg({
    'fare': [q_25, percentile_25, lambda_25, lambda x: x.quantile(.25)]
})

fare                                    
                  25% percentile_25 lambda_25% <lambda_0>
embark_town                                              
Cherbourg    13.69795      13.69795   13.69795   13.69795
Queenstown    7.75000       7.75000    7.75000    7.75000
Southampton   8.05000       8.05000    8.05000    8.05000

In [ ]:
def count_nulls(s):
  return s.size - s.count()

df.groupby(['deck']).agg({
    'embark_town': ['count', 'nunique', 'size', count_nulls, set]
})

embark_town                                                               
           count nunique size count_nulls                                   set
deck                                                                           
A             15       2   15           0              {Southampton, Cherbourg}
B             45       2   47           2         {nan, Southampton, Cherbourg}
C             59       3   59           0  {Southampton, Cherbourg, Queenstown}
D             33       2   33           0              {Southampton, Cherbourg}
E             32       3   32           0  {Southampton, Cherbourg, Queenstown}
F             13       3   13           0  {Southampton, Cherbourg, Queenstown}
G              4       1    4           0                         {Southampton}

In [ ]:
def top_10_sum(x):
  return x.nlargest(10).sum()

def bottom_10_sum(x):
  return x.nsmallest(10).sum()

df.groupby(['class']).agg({
    'fare': [top_10_sum, bottom_10_sum]
})

fare              
       top_10_sum bottom_10_sum
class                          
First   3361.2584      108.3709
Second   622.2376       42.0000
Third    656.3374       36.1291

If you have a scenario where you want to run multiple aggregations across columns, then you may want to use the groupby combined with apply as described in this stack overflow answer.
https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns/47103408#47103408

In [ ]:
def my_summary(x):
  result = {
      'fare_sum': x['fare'].sum(),
      'fare_mean': x['fare'].mean(),
      'fare_range': x['fare'].max() - x['fare'].min()
  }
  return pd.Series(result).round(0)

df.groupby(['class']).apply(my_summary)

,fare_sum,fare_mean,fare_range
class,,,
First,18177.0,84.0,512.0
Second,3802.0,21.0,74.0
Third,6715.0,14.0,70.0


Using apply with groupy gives maximum flexibility over all aspects of the results. However, there is a downside. The apply function is slow so this approach should be used sparingly.

In [ ]:
df.groupby(['embark_town', 'class']).agg({
    'fare': ['sum']
}).assign(
  pct_total=lambda x: x / x.sum()
)

fare pct_total
                          sum          
embark_town class                      
Cherbourg   First   8901.0750  0.311947
            Second   431.0917  0.015108
            Third    740.1295  0.025939
Queenstown  First    180.0000  0.006308
            Second    37.0500  0.001298
            Third    805.2043  0.028219
Southampton First   8936.3375  0.313183
            Second  3333.7000  0.116833
            Third   5169.3613  0.181165

In [ ]:
df.groupby(['embark_town', 'class']) \
  .agg({'fare': 'sum'}) \
  .groupby(level=0) \
  .cumsum() # soma cumulativa

fare
embark_town class             
Cherbourg   First    8901.0750
            Second   9332.1667
            Third   10072.2962
Queenstown  First     180.0000
            Second    217.0500
            Third    1022.2543
Southampton First    8936.3375
            Second  12270.0375
            Third   17439.3988

In [ ]:
df.groupby(['embark_town']).agg({'fare': 'sum'})

,fare
embark_town,
Cherbourg,10072.2962
Queenstown,1022.2543
Southampton,17439.3988


# Aggregation Crosstab

In [ ]:
pd.crosstab(df['embark_town'], 
            df['class'], 
            values=df['fare'], 
            aggfunc='sum', 
            normalize=True)

class,First,Second,Third
embark_town,,,
Cherbourg,0.311947,0.015108,0.025939
Queenstown,0.006308,0.001298,0.028219
Southampton,0.313183,0.116833,0.181165


# Aggregation Pivot Table

In [ ]:
pd.pivot_table(data=df,
               index=['embark_town'],
               columns=['class'],
               aggfunc={'fare': ['mean', 'sum']})

fare                                                       
                   mean                              sum                      
class             First     Second      Third      First     Second      Third
embark_town                                                                   
Cherbourg    104.718529  25.358335  11.214083  8901.0750   431.0917   740.1295
Queenstown    90.000000  12.350000  11.183393   180.0000    37.0500   805.2043
Southampton   70.364862  20.327439  14.644083  8936.3375  3333.7000  5169.3613

In [ ]:
df = pd.read_excel('https://pbpython.com/extras/sales-funnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [ ]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won", "pending", "presented", "declined"], inplace=True)

In [ ]:
pd.pivot_table(df, index=['Name'])

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000


In [ ]:
pd.pivot_table(df, index=['Name', 'Rep', 'Manager'])

,,,Account,Price,Quantity
Name,Rep,Manager,,,
Barton LLC,John Smith,Debra Henley,740150,35000,1.000000
"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,737550,35000,1.000000
Herman LLC,Cedric Moss,Fred Anderson,141962,65000,2.000000
Jerde-Hilpert,John Smith,Debra Henley,412290,5000,2.000000
"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,307599,7000,3.000000
Keeling LLC,Wendy Yule,Fred Anderson,688981,100000,5.000000
Kiehn-Spinka,Daniel Hilton,Debra Henley,146832,65000,2.000000
Koepp Ltd,Wendy Yule,Fred Anderson,729833,35000,2.000000
Kulas Inc,Daniel Hilton,Debra Henley,218895,25000,1.500000


In [ ]:
pd.pivot_table(df, index=["Manager", "Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

In [ ]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

In [ ]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [ ]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=[np.mean, len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               columns=["Product"],
               values=["Price"], 
               aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               columns=["Product"],
               values=["Price"], 
               aggfunc=[np.sum],
               fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep"], 
               columns=["Product"],
               values=["Price", "Quantity"], 
               aggfunc=[np.sum],
               fill_value=0)

sum              ...                  
                              Price              ... Quantity         
Product                         CPU Maintenance  ...  Monitor Software
Manager       Rep                                ...                  
Debra Henley  Craig Booker    65000        5000  ...        0        1
              Daniel Hilton  105000           0  ...        0        1
              John Smith      35000        5000  ...        0        0
Fred Anderson Cedric Moss     95000        5000  ...        0        1
              Wendy Yule     165000        7000  ...        2        0

[5 rows x 8 columns]

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Rep", "Product"], 
               values=["Price", "Quantity"], 
               aggfunc=[np.sum, np.mean],
               fill_value=0,
               margins=True)

sum  ...      mean
                                          Price  ...  Quantity
Manager       Rep           Product              ...          
Debra Henley  Craig Booker  CPU           65000  ...  1.000000
                            Maintenance    5000  ...  2.000000
                            Software      10000  ...  1.000000
              Daniel Hilton CPU          105000  ...  2.000000
                            Software      10000  ...  1.000000
              John Smith    CPU           35000  ...  1.000000
                            Maintenance    5000  ...  2.000000
Fred Anderson Cedric Moss   CPU           95000  ...  1.500000
                            Maintenance    5000  ...  1.000000
                            Software      10000  ...  1.000000
              Wendy Yule    CPU          165000  ...  3.500000
                            Maintenance    7000  ...  3.000000
                            Monitor        5000  ...  2.000000
All                                      522000  ...  1.764706

[14 rows x 4 columns]

In [ ]:
pd.pivot_table(df, 
               index=["Manager", "Status"], 
               values=["Price"], 
               aggfunc=[np.sum],
               fill_value=0,
               margins=True)

sum
                          Price
Manager       Status           
Debra Henley  won         65000
              pending     50000
              presented   50000
              declined    70000
Fred Anderson won        172000
              pending      5000
              presented   45000
              declined    65000
All                      522000

In [ ]:
table = pd.pivot_table(df,
               index=["Manager", "Status"],
               columns=["Product"],
               values=["Quantity", "Price"],
               aggfunc={
                   "Quantity": len,
                   "Price": np.sum
               },
               fill_value=0)
table

Price              ... Quantity         
Product                     CPU Maintenance  ...  Monitor Software
Manager       Status                         ...                  
Debra Henley  won         65000           0  ...        0        0
              pending     40000       10000  ...        0        0
              presented   30000           0  ...        0        2
              declined    70000           0  ...        0        0
Fred Anderson won        165000        7000  ...        0        0
              pending         0        5000  ...        0        0
              presented   30000           0  ...        1        1
              declined    65000           0  ...        0        0

[8 rows x 8 columns]

In [ ]:
table.query('Manager == ["Debra Henley"]')

Price                      ...    Quantity                 
Product                   CPU Maintenance Monitor  ... Maintenance Monitor Software
Manager      Status                                ...                             
Debra Henley won        65000           0       0  ...           0       0        0
             pending    40000       10000       0  ...           2       0        0
             presented  30000           0       0  ...           0       0        2
             declined   70000           0       0  ...           0       0        0

[4 rows x 8 columns]

In [ ]:
table.query('Status == ["pending", "won"]')

Price                      ...    Quantity                 
Product                   CPU Maintenance Monitor  ... Maintenance Monitor Software
Manager       Status                               ...                             
Debra Henley  won       65000           0       0  ...           0       0        0
              pending   40000       10000       0  ...           2       0        0
Fred Anderson won      165000        7000       0  ...           1       0        0
              pending       0        5000       0  ...           1       0        0

[4 rows x 8 columns]

# Aggregation Grouper

# Style

In [ ]:
df = pd.read_excel('https://github.com/chris1610/pbpython/blob/master/data/2018_Sales_Total.xlsx?raw=true')
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2018-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2018-01-01 23:26:55


In [ ]:
(df.groupby('name')['ext price']
   .agg(['mean', 'sum'])
   .style.format('${0:,.2f}'))

,mean,sum
name,,
Barton LLC,"$1,334.62","$109,438.50"
"Cronin, Oberbrunner and Spencer","$1,339.32","$89,734.55"
"Frami, Hills and Schmidt","$1,438.47","$103,569.59"
"Fritsch, Russel and Anderson","$1,385.37","$112,214.71"
"Halvorson, Crona and Champlin","$1,206.97","$70,004.36"
Herman LLC,"$1,336.53","$82,865.00"
Jerde-Hilpert,"$1,265.07","$112,591.43"
"Kassulke, Ondricka and Metz","$1,350.80","$86,451.07"
Keeling LLC,"$1,363.98","$100,934.30"


In [ ]:
monthly_sales = df.groupby([pd.Grouper(key='date', freq='M')])['ext price'].agg(['sum']).reset_index()
monthly_sales['pct_of_total'] = monthly_sales['sum'] / df['ext price'].sum()
monthly_sales.head()

,date,sum,pct_of_total
0,2018-01-31,185361.66,0.091818
1,2018-02-28,146211.62,0.072426
2,2018-03-31,203921.38,0.101012
3,2018-04-30,174574.11,0.086475
4,2018-05-31,165418.55,0.081940


In [ ]:
format_dict = {
    'sum': '${0:,.0f}',
    'date': '{:%m-%Y}',
    'pct_of_total': '{:.2%}'
}

(monthly_sales
  .style
  .format(format_dict)
  .hide_index())

date,sum,pct_of_total
01-2018,"$185,362",9.18%
02-2018,"$146,212",7.24%
03-2018,"$203,921",10.10%
04-2018,"$174,574",8.65%
05-2018,"$165,419",8.19%
06-2018,"$174,089",8.62%
07-2018,"$191,662",9.49%
08-2018,"$153,779",7.62%
09-2018,"$168,443",8.34%
10-2018,"$171,495",8.49%


In [ ]:
(monthly_sales
  .style
  .format(format_dict)
  .hide_index()
  .highlight_max(color='lightgreen')
  .highlight_min(color='#cd4f39'))

date,sum,pct_of_total
01-2018,"$185,362",9.18%
02-2018,"$146,212",7.24%
03-2018,"$203,921",10.10%
04-2018,"$174,574",8.65%
05-2018,"$165,419",8.19%
06-2018,"$174,089",8.62%
07-2018,"$191,662",9.49%
08-2018,"$153,779",7.62%
09-2018,"$168,443",8.34%
10-2018,"$171,495",8.49%


In [ ]:
(monthly_sales
  .style
  .format(format_dict)
  .background_gradient(subset=['sum'], cmap='BuGn'))

,date,sum,pct_of_total
0,01-2018,"$185,362",9.18%
1,02-2018,"$146,212",7.24%
2,03-2018,"$203,921",10.10%
3,04-2018,"$174,574",8.65%
4,05-2018,"$165,419",8.19%
5,06-2018,"$174,089",8.62%
6,07-2018,"$191,662",9.49%
7,08-2018,"$153,779",7.62%
8,09-2018,"$168,443",8.34%
9,10-2018,"$171,495",8.49%


In [ ]:
(monthly_sales
  .style
  .format(format_dict)
  .hide_index()
  .bar(color='#FFA07A', vmin=100000, subset=['sum'], align='zero')
  .bar(color='lightgreen', vmin=0, subset=['pct_of_total'], align='zero')
  .set_caption('2018 Sales Performance'))

date,sum,pct_of_total
01-2018,"$185,362",9.18%
02-2018,"$146,212",7.24%
03-2018,"$203,921",10.10%
04-2018,"$174,574",8.65%
05-2018,"$165,419",8.19%
06-2018,"$174,089",8.62%
07-2018,"$191,662",9.49%
08-2018,"$153,779",7.62%
09-2018,"$168,443",8.34%
10-2018,"$171,495",8.49%


# Indexing

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [25]:
df = sns.load_dataset("titanic")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [28]:
# label based
df.loc[10:15, ['pclass', 'sex', 'survived']]

,pclass,sex,survived
10,3,female,1
11,1,female,1
12,3,male,0
13,3,male,0
14,3,female,0
15,2,female,1


In [30]:
# index based
df.iloc[10:15]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True
12,0,3,male,20.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
13,0,3,male,39.0,1,5,31.2750,S,Third,man,True,NaN,Southampton,no,False
14,0,3,female,14.0,0,0,7.8542,S,Third,child,False,NaN,Southampton,no,True


In [31]:
first = df.groupby('embark_town')[['fare', 'class']].first()
first.head()

,fare,class
embark_town,,
Cherbourg,71.2833,First
Queenstown,8.4583,Third
Southampton,7.2500,Third


In [32]:
first.iloc[10:15] # not reliable

,fare,class
embark_town,,


In [33]:
first.loc[['Cherbourg'], ['fare', 'class']] # safer and easy to understand

,fare,class
embark_town,,
Cherbourg,71.2833,First


In [36]:
f = pd.DataFrame({'a':[1,2,3,4,5], 'b':[10,20,30,40,50]})
f

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [37]:
f.loc[f['a'] <= 3, 'b'] /= 10
f 

,a,b
0,1,1.0
1,2,2.0
2,3,3.0
3,4,40.0
4,5,50.0


In [39]:
hdf = df.set_index(['embark_town', 'pclass', 'sex']).sort_index()
hdf[['fare', 'alone', 'survived']].head()

fare  alone  survived
embark_town pclass sex                              
Cherbourg   1      female   71.2833  False         1
                   female  146.5208  False         1
                   female   76.7292  False         1
                   female   28.7125   True         0
                   female   27.7208   True         1

In [42]:
hdf.loc[['Queenstown'], ['fare', 'survived']].head()

fare  survived
embark_town pclass sex                    
Queenstown  1      female  90.00         1
                   male    90.00         0
            2      female  12.35         1
                   female  12.35         1
                   male    12.35         0

In [44]:
hdf.loc[(['Queenstown'], [2]), ['fare', 'survived']].head()

fare  survived
embark_town pclass sex                    
Queenstown  2      female  12.35         1
                   female  12.35         1
                   male    12.35         0

In [48]:
hdf.loc[(['Queenstown'], [2], ['male']), ['fare', 'survived']].head()

,,,fare,survived
embark_town,pclass,sex,,
Queenstown,2,male,12.35,0


In [46]:
# Para melhor buscar por indice
hdf.loc[pd.IndexSlice[:, :, ['male']], ['fare', 'survived']].head()

fare  survived
embark_town pclass sex                    
Cherbourg   1      male  27.7208         0
                   male  82.1708         0
                   male  61.9792         0
                   male  27.7208         0
                   male  34.6542         0

# Method Chaining

R:


```
jack_jill %>%
    went_up("hill") %>%
    fetch("water") %>%
    fell_down("jack") %>%
    broke("crown") %>%
    tumble_after("jill")
```

Pandas:


```
jack_jill = pd.DataFrame()
(jack_jill.pipe(went_up, 'hill')
    .pipe(fetch, 'water')
    .pipe(fell_down, 'jack')
    .pipe(broke, 'crown')
    .pipe(tumble_after, 'jill')
)
```

In [50]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [58]:
def my_function(df):
  print(df.shape)
  return df

(df[['pclass', 'sex', 'age', 'fare', 'embark_town']]
  .dropna()
  .assign(is_male=lambda x: x.sex == 'male')
  .query('10 < fare < 50')
  .groupby(["embark_town", "pclass"])
  .agg({
      'is_male': sum
  })
  .pipe(my_function)
  .head())

(8, 1)


is_male
embark_town pclass         
Cherbourg   1            13
            2             8
            3             6
Queenstown  2             1
            3             5